[View in Colaboratory](https://colab.research.google.com/github/oasispapa/hello-world/blob/master/Chapter13.ipynb)

# Chapter 13. 연산자 오버로딩: 제대로 하기

1. 배경 및 개념
 * 연산자란? 
  >- 수학연산과 유사한 연산자의 집합. <BR>
  > 예) 산술연산(+-), 비교연산(>=), 논리연산(T/F), 비트연산(and or xor..), 복합할당연산 (+=), 멤버와 포인트 연산(*) 등이 있고 서로간 우선순위가 있음. 
   
 * 오버로드란?
   >- **함수 오버로드** : 같은 함수 이름을 가지고 있으나 매개변수 타입이나 개수, 리턴타입등을 달리 구현하여 여러개의 서브프로그램을 생성하는 기법. <BR>
   예) doTask() , doTask(object O)   
   >- **연산자 오버로드** : 객체 지향 프로그래밍에서 다형성의 특정 경우로 다른 연산자들이 함수 인자를 통해 구현 할 때를 말한다. 연산자 오버로딩은 일반적으로 언어, 프로그래머, 또는 두 가지 모두에 의해 정의된다. (위키백과). (한마디로 +가 숫자더할때나 문자더할때 쓰일 수 있는 것은 연산자 오버로드를 했기 때문임.)

2. 주의점(?) & 파이썬에서 연산자 오버로드란?
 * 연산자 오버로딩을 혐오하는 사람이 많다. <BR>
 -> 왜? 남용, 혼란, 버그, 병목발생.. 신경쓸게 많다. <BR>예) +연산에서  교환법칙이 수에서는 성립하지만, 문자열에서는 불가하다.   
 * 1960년대 ALGOL 68 연산자 오버로딩 허용. <BR>
   1980년대 에이다 연산자 오버로딩 지원. (새로운 연산자 정의 허용  안 함) <BR>
   1990년대 자바는 연산자 오버로딩 불포함<BR>
   2001년 c#은 연산자 오버로딩 포함. <BR>
 * 그래서 **파이썬**에서는 아래와 같은 제한을 둔다. 
 >- 내장 자료형(리스트, 튜플, 딕셔너리)에 대한 연산자는 오버로딩할 수 없다. 
 >- is, and, or, not 연산자는 오버로딩할 수 없다. 
 >- 새로운 연산자를 생성할 수 없으며, 기존 연산자를 오버로딩만 할 수 있다. 
 * 최근 과학 계산 분야에 파이썬이 널리 보급된 이유는 사용하기 쉬운 고수준 언어로 연산자를 오버로딩하기 때문일 것이다.
 * 반면, 성능과 안전이 가장 중요한 저수준 시스템 언어에서는 연산자 오버로딩을 허용하지 않는 것이 타당하다. 
    
2. 연산자 종류
 * 단항 연산자 : 
 >- **-(__neg__) 단항 산술 부정** : x가 -2면, -x는 2
 >- **+(__pos__) 단항 산술 덧셈** : 일반적으로 x와 +x는 동일하지만 아닌경우 있음. 
 >- **~(__invert__) 비트 반전** : ~x는 -(x+1). x가 2면 ~x는 -3.  (이와같은 정수형 외에서는 부정한 결과를 반환. SQL 에서 사용한다면 WHERE 구를 부정한 결과를 리턴)
 * 중위 연산자 :  
  >- 앞서 본 예제 Vector 클래스 안에서 연산자 오버로딩을 통해 중위 연산자 메서드를 살펴본다.
 >- 중위 연산자 메서드 <br>
     예)  +  -  *  /  //  %  divmod()  **  @  &  |  ^  <<  >> 
 >- 비교 연산자 <br>
     예) >  <  >=  <=  ==  !=
 >- 복합 할당 연산자 <br>
     예) +=  *=
 * 기타 연산자 : 함수호출 '()', 속성접근 '.', 항목접근/슬라이싱 '[]' -> 파이썬에서는 연산자로 구현되어있지만 이 장에서는 다루지 않음. 
 


# 단항 연산자
구현하기 쉽다. <br>
단지 self 인수 하나를 받는 적절한 특별 메서드를 구현하면 된다. <br>
다만 **언제나 새로운 객체를 반환**해야한다는 연산자 핵심규칙을 지켜야한다. <br>


In [0]:
# 예제 10-16에서 이미 구현해봤던 클래스
from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools  # <1>


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    def angle(self, n):  # <2>
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):  # <3>
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())  # <4>
            outer_fmt = '<{}>'  # <5>
        else:
            coords = self
            outer_fmt = '({})'  # <6>
        components = (format(c, fmt_spec) for c in coords)  # <7>
        return outer_fmt.format(', '.join(components))  # <8>

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    
    
    # 이번 장에서 추가한 코드 
    # 단항 연산자
    def __neg__(self):
        return Vector(-x for x in self)

    def __pos__(self):
        return Vector(self)
    
    
    # 중위 연산자    
    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs) 
        except TypeError:
            return NotImplemented
            # return TypeError
    
    def __radd__(self, other):
        return self + other
            
    
    def __mul__(self, scalar):
        if isinstance(scalar, numbers.Real):
            return Vector(n * scalar for n in self)
        else:
            return NotImplemented
    
    def __rmul__(self, scalar):
        return self * scalar
    
    def __matmul__(self, other):
        try:
            return sum(a * b for a, b in zip(self, other))
        except TypeError:
            return NotImplemented
        
    def __rmatmul__(self, other):
        return self @ other
    
    def __eq__(self, other):
        #if isinstance(other, Vector):
        return (len(self) == len(other) and all(a == b for a, b in zip(self, other)))
        #else:
        #    return NotImplemented

In [26]:
v = Vector([1, 2, 3])
# 단항 산술 부정
print (-v)
# 단항 산술 덧셈
print (+v)

(-1.0, -2.0, -3.0)
(1.0, 2.0, 3.0)


**x와 +x 가 동일하지 않은 경우 <br>**
사실 파이썬에서는 거의 항상 똑같다. <br>
표준 라이브러리 안에서 x와 +x가 다른 두 가지  사례 (x != +x)

In [27]:
# 사례 1 : decimal.Decimal
import decimal
ctx = decimal.getcontext()
# print (type (ctx))
ctx.prec = 10
one_third = decimal.Decimal('1') / decimal.Decimal('3')
print (one_third)

0.3333333333


In [28]:
one_third == +one_third

True

In [29]:
ctx.prec = 3
one_third == +one_third

False

In [30]:
+one_third

Decimal('0.333')

In [40]:
one_third

Decimal('0.3333333333')

=> +one_third 표현식이 **나타낼때마다** one_third 의 값을 이용해서 Decimal 객체를 새로 만드는데, 이때 현재의 산술 콘텍스트를 사용한다. 

In [32]:
# 사례 2 : collections.Counter
from collections import Counter
ct = Counter('abracadabra')
ct


Counter({'a': 5, 'b': 2, 'c': 1, 'd': 1, 'r': 2})

In [33]:
ct['r'] = -3
ct['d'] = 0
ct

Counter({'a': 5, 'b': 2, 'c': 1, 'd': 0, 'r': -3})

In [10]:
+ct

Counter({'a': 5, 'b': 2, 'c': 1})

In [34]:
Counter('aaa') + Counter('bb')

Counter({'a': 3, 'b': 2})

#중위연산자
class Vector 에서의  + 및 * 오버로딩

In [5]:
v1 = Vector([3, 4, 5])
v2 = Vector([6, 7, 8])
v1 + v2

Vector([9.0, 11.0, 13.0])

In [35]:
# cf) 인터넷으로 찾은 벡터 합 (array)
import numpy as np

va1 = np.array([3, 4, 5])
va2 = np.array([6, 7, 8])
va1 + va2

array([ 9, 11, 13])

In [37]:
v1 = Vector([3, 4, 5, 6])
v3 = Vector([1, 2])
v1 + v3

Vector([4.0, 6.0, 5.0, 6.0])

In [38]:
v1 = Vector([3, 4, 5])
v1 + (10, 20, 30)

Vector([13.0, 24.0, 35.0])

In [41]:
# Vector class에 역순 연산자 메서드가 없으면 오류 발생 (그림 13-1 참고. 요약: 먼저 정방향 메서드 실행하고, NotImplemented 가 반환되면 역순 메서드를 실행)
v1 = Vector([3, 4, 5])
(10, 20, 30) + v1 

Vector([13.0, 24.0, 35.0])

In [43]:
v1 = Vector([1, 2, 3])
print (v1 * 10)
print (11 * v1)
print (v1 * True)
print (v1 * False)

(10.0, 20.0, 30.0)
(11.0, 22.0, 33.0)
(1.0, 2.0, 3.0)
(0.0, 0.0, 0.0)


In [44]:
from fractions import Fraction
print (v1 * Fraction(1, 3))
print (Fraction(1, 3))

(0.3333333333333333, 0.6666666666666666, 1.0)
1/3


**중위 연산자 메서드 명**
<table>
    <tr>
        <th>연산자</th>
        <th>정방향</th>
        <th>역순</th>
        <th>인플레이스</th>
        <th>설명</th>
    </tr>
    <tr>
        <td>+</td>
        <td>\__add\__()</td>
        <td>\__radd\__()</td>
        <td>\__iadd\__()</td>
        <td>덧셈이나 연결</td>
    </tr>
    <tr>
        <td>-</td>
        <td>\__sub\__()</td>
        <td>\__rsub\__()</td>
        <td>\__isub\__()</td>
        <td>뺄셈</td>
    </tr>
    <tr>
        <td>\*</td>
        <td>\__mul\__()</td>
        <td>\__rmul\__()</td>
        <td>\__imul\__()</td>
        <td>곱셈이나 반복</td>
    </tr>
    <tr>
        <td>/</td>
        <td>\__truediv\__()</td>
        <td>\__rtruediv\__()</td>
        <td>\__itruediv\__()</td>
        <td>참 나눗셈</td>
    </tr>
    <tr>
        <td>//</td>
        <td>\__floordiv\__()</td>
        <td>\__rfloordiv\__()</td>
        <td>\__ifloordiv\__()</td>
        <td>플로어 나눗셈</td>
    </tr>
    <tr>
        <td>%</td>
        <td>\__mod\__()</td>
        <td>\__rmod\__()</td>
        <td>\__imod\__()</td>
        <td>모듈로(나머지) 연산</td>
    </tr>
    <tr>
        <td>divmod()</td>
        <td>\__divmod\__()</td>
        <td>\__rdivmod\__()</td>
        <td>\__idivmod\__()</td>
        <td>플로어 나눗셈의 몫과 나머지를 튜플로 반환한다.</td>
    </tr>
    <tr>
        <td>**, pow()</td>
        <td>\__pow\__()</td>
        <td>\__rpow\__()</td>
        <td>\__ipow\__()</td>
        <td>누승</td>
    </tr>
    <tr>
        <td>@</td>
        <td>\__matmul\__()</td>
        <td>\__rmatmul\__()</td>
        <td>\__imatmul\__()</td>
        <td>행렬 곱셈 (파이썬 3.5)</td>
    </tr>
    <tr>
        <td>&</td>
        <td>\__and\__()</td>
        <td>\__rand\__()</td>
        <td>\__iand\__()</td>
        <td>비트단위 곱 (bitwise and)</td>
    </tr>
    <tr>
        <td>|</td>
        <td>\__or\__()</td>
        <td>\__rorv\__()</td>
        <td>\__ior\__()</td>
        <td>비트단위 합 (bitwise or)</td>
    </tr>
    <tr>
        <td>^</td>
        <td>\__xor\__()</td>
        <td>\__rxor\__()</td>
        <td>\__ixor\__()</td>
        <td>비트단위 배타합 (bitwise xor)</td>
    </tr>
    <tr>
        <td> << </td>
        <td>\__lshift\__()</td>
        <td>\__rlshift\__()</td>
        <td>\__ishift\__()</td>
        <td>비트단위 왼쪽 시프트</td>
    </tr>
    <tr>
        <td> >> </td>
        <td>\__rshift\__()</td>
        <td>\__rrshift\__()</td>
        <td>\__irshift\__()</td>
        <td>비트단위 오른쪽 시프트</td>
    </tr>
 </table>

In [45]:
# 행렬 곱셈 @ 중위 연산자 
va = Vector([1, 2, 3])
vz = Vector([5, 6, 7])
va @ vz

38.0

**비교 연산자**<br>
앞에 설명한 것과 비슷 (정방향 메서드 실행 -> NotImplemented 반환되면 역순 메서드 실행..)
<br>
<table>
    <tr>
        <th>종류</th>
        <th>중위연산자</th>
        <th>정방향</th>
        <th>역순</th>
        <th>(역순 실패시) 기본 처리</th>
    </tr>
    <tr>
        <td rowspan=2>동치성</td>
        <td>a == b</td>
        <td>a.\__eq\__(b)</td>
        <td>b.\__eq\__(a)</td>
        <td>id(a) == id(b) 를 반환한다.</td>
    </tr>
    <tr>
        <td>a != b</td>
        <td>a.\__ne\__(b)</td>
        <td>b.\__ne\__(a)</td>
        <td>not (a == b) 를 반환한다.</td>
    </tr>
    <tr>
        <td rowspan=4>순서</td>
        <td>a > b</td>
        <td>a.\__gt\__(b)</td>
        <td>b.\__lt\__(a)</td>
        <td>TypeError를 발생시킨다.</td>
    </tr>
    <tr>
        <td>a < b</td>
        <td>a.\__lt\__(b)</td>
        <td>b.\__gt\__(a)</td>
        <td>TypeError를 발생시킨다.</td>
    </tr>
    <tr>
        <td>a >= b</td>
        <td>a.\__ge\__(b)</td>
        <td>b.\__le\__(a)</td>
        <td>TypeError를 발생시킨다.</td>
    </tr>
    <tr>
        <td>a <= b</td>
        <td>a.\__le\__(b)</td>
        <td>b.\__ge\__(a)</td>
        <td>TypeError를 발생시킨다.</td>
    </tr>
</table>

In [49]:
va = Vector([1.0, 2.0, 3.0])
t3 = (1, 2, 3)
print (va == t3)
print (id(va), id(t3))

# => 동일한 숫자 요소를 가진 두 Vector 객체를 동일하다고 판단하도록 구현 가능함. Vector 끼리의 비교인지 체크하려면 isinstance 체크로직 넣어줌됨. 

True
140047035945984 140047035676352


In [47]:
va = Vector([1.0, 2.0, 3.0])
vb = Vector(range(1, 4))
va == vb

True

**복합 할당 연산자** <br>
a += b 는 a = a + b 와 동일하게 \__add\__() 메서드를 호출. <br>
하지만, \__iadd\__() 와 같은 인플레이스 연산자 메서드를 정의한 경우 iadd 가 호출됨.<br>
그러나, 새로운 객체를 생성하지 않고 self 를 직접 변경하게 되므로 Vector 같은 불변 자료형에서는 구현하면 안 된다. 

In [50]:
v1 = Vector([1, 2, 3])
v1 += Vector([4, 5, 6])
print (v1)

# => 복합할당이 불변 타깃을 처리할 때는 객체를 새로 생성하고 다시 바인딩한다. 

(5.0, 7.0, 9.0)


In [0]:
# BigoCage 를 통한 iadd 오버로딩해보기

import abc
import random


class Tombola(abc.ABC):

    @abc.abstractmethod
    def load(self, iterable):
        """Add items from an iterable."""

    @abc.abstractmethod
    def pick(self):
        """Remove item at random, returning it.

        This method should raise `LookupError` when the instance is empty.
        """

    def loaded(self):
        """Return `True` if there's at least 1 item, `False` otherwise."""
        return bool(self.inspect())


    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))
    

class BingoCage(Tombola):

    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):
        self.pick()
        
        
    
# 이번 장에서 추가한 코드 
class AddableBingoCage(BingoCage):
    
    def __add__(self, other):
        if isinstance(other, Tombola):
            return AddableBingoCage(self.inspect() + other.inspect())
        else:
            return NotImplemented

    def __iadd__(self, other):
        if isinstance(other, Tombola):
            other_iterable = other.inspect()
        else:
            try:
                other_iterable = iter(other)
            except TypeError:
                self_cls = type(self).__name__
                msg = "right operand in += must be {!r} or an iterable"
                raise TypeError(msg.format(self_cls))
        self.load(other_iterable) 
        return self


In [53]:
vowels = 'AEIOU'
globe = AddableBingoCage(vowels)
globe2 = AddableBingoCage('XYZ')
globe_org = globe
print (globe_org.inspect())
# 복합할당
globe += globe2
print (globe.inspect())
print (globe_org.inspect())
# 중위연산메서드
globe2 = AddableBingoCage('')
globe3 = globe_org + globe2

print (globe3.inspect())

print (globe is globe_org)
print (globe is globe3)

# 복합할당 연산자는 새로운 객체를 만들지 않고 자신에게 결과값을 담는다. 
# +연산자는 결과값의 자료형에 대해 혼란스러울 수 있지만 += 연산자는 내용의 갱신이므로 연산 결과 자료형이 명확한 이점이 있다. 

('A', 'E', 'I', 'O', 'U')
('A', 'E', 'I', 'O', 'U', 'X', 'Y', 'Z')
('A', 'E', 'I', 'O', 'U', 'X', 'Y', 'Z')
('A', 'E', 'I', 'O', 'U', 'X', 'Y', 'Z')
True
False


In [70]:
a = ''
if a:
    print('11')
else:
    print('22')

22
